<link rel="preconnect" href="https://fonts.gstatic.com">
<link href="https://fonts.googleapis.com/css2?family=Fira+Code&display=swap" rel="stylesheet">

### License

<p style="font-family: 'Fira Code', monospace; font-size: 1.2rem">
Copyright 2021 Xilinx, Inc.<br><br>
Licensed under the Apache License, Version 2.0 (the "License");<br>
you may not use this file except in compliance with the License.<br><br>
You may obtain a copy of the License at <a href="http://www.apache.org/licenses/LICENSE-2.0"?>http://www.apache.org/licenses/LICENSE-2.0</a><br><br>
Unless required by applicable law or agreed to in writing, software<br>
distributed under the License is distributed on an "AS IS" BASIS,<br>
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.<br>
See the License for the specific language governing permissions and<br>
limitations under the License.<br>
</p>


# ChipScoPy System Monitor Example


<img src="../img/api_overview.png" width="500" align="left">

## Description


This demo shows how to take and display measurements with the System Monitor.


## Requirements
- Local or remote Xilinx Versal board, such as a VCK190
- Xilinx hw_server 2021.2 installed and running
- Xilinx cs_server 2021.2 installed and running
- Python 3.8 or greater installed
- ChipScoPy 2021.2 installed
- Jupyter notebook support installed - Please do so, using the command `pip install chipscopy[jupyter]`

## 1 - Initialization: Imports and File Paths

After this step,
- Required functions and classes are imported
- URL paths are set correctly
- File paths to example files are set correctly

In [1]:
import os
import time
from chipscopy import get_design_files
from chipscopy import __version__, dm
from chipscopy import create_session, report_versions

In [2]:
# Make sure to start the hw_server and cs_server prior to running.
# Specify locations of the running hw_server and cs_server below.
# The default is localhost - but can be other locations on the network.
CS_URL = os.getenv("CS_SERVER_URL", "TCP:localhost:3042")
HW_URL = os.getenv("HW_SERVER_URL", "TCP:localhost:3121")

# The get_design_files() function tries to find the PDI and LTX files. In non-standard
# configurations, you can put the path for PROGRAMMING_FILE and PROBES_FILE below.
design_files = get_design_files("vck190/production/chipscopy_ced")

PROGRAMMING_FILE = design_files.programming_file
PROBES_FILE = design_files.probes_file

print(f"HW_URL: {HW_URL}")
print(f"CS_URL: {CS_URL}")
print(f"PROGRAMMING_FILE: {PROGRAMMING_FILE}")
print(f"PROBES_FILE:{PROBES_FILE}")

HW_URL: TCP:localhost:3121
CS_URL: TCP:localhost:3042
PROGRAMMING_FILE: C:\wrk\chipscopy\chipscopy\examples\designs\vck190\production\chipscopy_ced\chipscopy_wrapper.pdi
PROBES_FILE:C:\wrk\chipscopy\chipscopy\examples\designs\vck190\production\chipscopy_ced\chipscopy_wrapper.ltx


## 2 - Create a session and connect to the hw_server and cs_server

The session is a container that keeps track of devices and debug cores.
After this step,
- Session is initialized and connected to server(s)
- Versions are detected and reported to stdout

In [3]:
session = create_session(cs_server_url=CS_URL, hw_server_url=HW_URL)
report_versions(session)

## 3 - Program the device with the example design

After this step,
- Device is programmed with the example programming file

In [4]:
# Typical case - one device on the board - get it.
device = session.devices.filter_by(family="versal").get()
device.program(PROGRAMMING_FILE)

Output()

## 4 - Discover Debug Cores

Debug core discovery initializes the chipscope server debug cores. This brings debug cores in the chipscope server online.

After this step,

- The cs_server is initialized and ready for use

In [5]:
device.discover_and_setup_cores(sysmon_scan=True)
print(f"System monitor setup and ready for use.")

System monitor setup and ready for use.


## 5 - Initialize System Monitor

Get reference to the system monitor and initialize all sensors. 

In [6]:
sysmon = device.sysmon_root.get()

print("Initializing sensors")
active_nodes = sysmon.initialize_sensors()

print("Refresh measurement schedule")
schedule = sysmon.refresh_measurement_schedule()

print("Sensors:")
for sensor in schedule.values():
    print(f"  {sensor}")
    
print("Done.")

Initializing sensors
Refresh measurement schedule
Sensors:
  VCCAUX
  VCC_SOC
  VCCAUX_PMC
  VCC_PMC
  VCC_PSFP
  VCC_PSLP
  VP_VN
Done.


## 6 - Register a listener for System Monitor Events

The SysMonNodeListener node_changed() will be called every 1000ms with updated system monitor values.

In [7]:
class SysMonNodeListener(dm.NodeListener):
    def node_changed(self, node, updated_keys):
        if "device_temp" in node.props:
            print(f"Device Temp: {node.props['device_temp']}")
        for supply_idx, named_sensor in schedule.items():
            supply = f"supply{supply_idx}"
            if supply in node.props:
                print(f"{named_sensor}: {node.props[supply]}")
        print()


node_listener = SysMonNodeListener()
session.chipscope_view.add_node_listener(node_listener)

sysmon.stream_sensor_data(1000)
print("Node listener added.")

Node listener added.


## 7 - Run measurement for 5 seconds

System Monitor will report results for 5 seconds then exit.

In [8]:
# Take measurements for 5 seconds then exit.

time_end = time.time() + 5

while time.time() < time_end:
    session.chipscope_view.run_events()
    time.sleep(0.1)

print("Measurement done.")

Device Temp: 33.3671875
VCCAUX: 1.500518798828125
VCC_SOC: 0.8150634765625
VCCAUX_PMC: 1.49456787109375
VCC_PMC: 0.79852294921875
VCC_PSFP: 0.794647216796875
VCC_PSLP: 0.799591064453125
VP_VN: 0.119476318359375

Device Temp: 33.8515625
VCCAUX: 1.50054931640625
VCC_SOC: 0.80902099609375
VCCAUX_PMC: 1.495025634765625
VCC_PMC: 0.798736572265625
VCC_PSFP: 0.797393798828125
VCC_PSLP: 0.798004150390625
VP_VN: 0.11920166015625

Device Temp: 33.84375
VCCAUX: 1.500457763671875
VCC_SOC: 0.798858642578125
VCCAUX_PMC: 1.495330810546875
VCC_PMC: 0.79815673828125
VCC_PSFP: 0.7969970703125
VCC_PSLP: 0.79632568359375
VP_VN: 0.11907958984375

Device Temp: 33.8125
VCCAUX: 1.50042724609375
VCC_SOC: 0.80499267578125
VCCAUX_PMC: 1.494903564453125
VCC_PMC: 0.797698974609375
VCC_PSFP: 0.795654296875
VCC_PSLP: 0.797576904296875
VP_VN: 0.119049072265625

Device Temp: 33.7890625
VCCAUX: 1.5018310546875
VCC_SOC: 0.8046875
VCCAUX_PMC: 1.496307373046875
VCC_PMC: 0.799102783203125
VCC_PSFP: 0.796875
VCC_PSLP: 0.798